In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [5]:
df = pd.read_csv("E:/BDT/5001/Kaggle/raw_data/train.csv")

In [6]:
df.shape

(357, 11)

In [7]:
#将bool类型的全部转化为0和1
df['is_free']=df['is_free'].astype('int')

In [8]:
df.isnull().sum(axis=0)

id                        0
playtime_forever          0
is_free                   0
price                     0
genres                    0
categories                0
tags                      0
purchase_date             2
release_date              0
total_positive_reviews    2
total_negative_reviews    2
dtype: int64

In [9]:
#修改成日期格式 并用现在的时间填补空缺行
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['release_date'] = pd.to_datetime(df['release_date'])
df['purchase_date'].fillna(datetime.now(),inplace = True)

In [10]:
#将购买日期拆分成年份与月份 并将之设置为新的两列
def extract_date(df,purchase_date):
    df['purchase_date' + "_year"] = df['purchase_date'].apply(lambda x: x.year)
    df["purchase_date" + "_month"] = df["purchase_date"].apply(lambda x: x.month)
extract_date(df,"purchase_date")                                     

In [11]:
#添加新的两列 分别为 1、购买游戏到迄今为止的时间    2、游戏发行到游戏购买之间的时间
duration = datetime.now()-df['purchase_date']
df['duration'] = duration
df['duration'] = df['duration'].dt.days 
df['duration_to_release'] = df['purchase_date']-df['release_date']
df['duration_to_release'] = df['duration_to_release'].dt.days

In [13]:
#填补reviews的空缺 用均值填补
df = df.fillna(value = {"total_positive_reviews":df["total_positive_reviews"].mean(),
                        "total_negative_reviews":df["total_negative_reviews"].mean()})

In [14]:
#添加新的一列——好评率，有些游戏好评差评均为0个，因此用0填补空缺的好评率
df['total_reviews'] = df['total_positive_reviews']+df['total_negative_reviews']
df['positive_reviews_percent'] = df['total_positive_reviews']/df['total_reviews']
df['positive_reviews_percent'].fillna(0,inplace = True)

In [15]:
df.isnull().sum(axis=0)

id                          0
playtime_forever            0
is_free                     0
price                       0
genres                      0
categories                  0
tags                        0
purchase_date               0
release_date                0
total_positive_reviews      0
total_negative_reviews      0
purchase_date_year          0
purchase_date_month         0
duration                    0
duration_to_release         0
total_reviews               0
positive_reviews_percent    0
dtype: int64

In [17]:
#将以下各列的数值控制在同一阈值内，将之做标准化处理
df["price"] = df[['price']].apply(lambda x: (x-np.mean(x))/(np.std(x)))
df["total_reviews"] = df[['total_reviews']].apply(lambda x: (x-np.mean(x))/(np.std(x)))
df["positive_reviews_percent"] = df[['positive_reviews_percent']].apply(lambda x: (x-np.mean(x))/(np.std(x)))
df["purchase_date_year"] = df[['purchase_date_year']].apply(lambda x: (x-np.mean(x))/(np.std(x)))
df["purchase_date_month"] = df[['purchase_date_month']].apply(lambda x: (x-np.mean(x))/(np.std(x)))
df["duration"] = df[['duration']].apply(lambda x: (x-np.mean(x))/(np.std(x)))
df["duration_to_release"] = df[['duration_to_release']].apply(lambda x: (x-np.mean(x))/(np.std(x)))

In [18]:
# 对于object类型的数据全部进行onehot编码
df_new1 = (df.iloc[:,4]).str.get_dummies(sep=',')
df = pd.concat((df,df_new1),axis=1)
df_new2 = (df.iloc[:,5]).str.get_dummies(sep=',')
df = pd.concat((df,df_new2),axis=1)
df_new3 = (df.iloc[:,6]).str.get_dummies(sep=',')
df = pd.concat((df,df_new3),axis=1)

In [19]:
df.drop(["genres","categories","tags","total_positive_reviews","total_negative_reviews","purchase_date","release_date"],axis=1,inplace=True)

In [20]:
df.shape

(357, 371)

In [23]:
df.to_csv("E:/BDT/5001/Kaggle/train_set.csv",index = False)